## GPT-2 Small on GSMK8 

The purpose of this is to measure the coherence of an LLM on the Cohence of solving math problems within GSMK8

# Model set up

In [1]:
# import the hugging face transformers library
from transformers import pipeline, GPT2LMHeadModel, Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset




In [2]:


# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained("gpt2")



In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load and preprocess the dataset
def preprocess_data(examples):
    inputs = [q + " Answer: " for q in examples['question']]
    targets = [a for a in examples['answer']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    # Setup the tokenizer outputs as labels for training
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load the datasets
train_dataset = load_dataset("GSM8K", 'main', split='train')
test_dataset = load_dataset("GSM8K", 'main', split='test')

# Apply preprocessing
encoded_train_dataset = train_dataset.map(preprocess_data, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [5]:


# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="steps",     # evaluate during training to observe progress
    save_strategy="epoch"            # save the model at the end of each epoch
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset
    
)

# Start training
trainer.train()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss
10,9.790100,10.962371
20,8.801400,9.479906
30,6.838700,6.418238
40,4.513900,3.750113
50,2.862200,2.389358
60,1.973400,2.060966
70,1.826600,1.789028
80,1.578900,1.622379
90,1.443400,1.478700
100,1.434300,1.419629


TrainOutput(global_step=2805, training_loss=1.2114767156182764, metrics={'train_runtime': 7766.3009, 'train_samples_per_second': 2.887, 'train_steps_per_second': 0.361, 'total_flos': 5857906065408000.0, 'train_loss': 1.2114767156182764, 'epoch': 3.0})

In [7]:
!ls

 Coherence_gpt2_small.ipynb	    Set-up		    'not my work'
 GPT2-small-reward-modeling.ipynb   __pycache__		    'old experiments'
'MDP classifier'		   'big experiments'	     results
 MDP.py				    connect_4_lookup.ipynb   temp.md
 README.md			    images
'RL gym coherence'		    my_dir


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Results

In [13]:


# Load your fine-tuned model and tokenizer from the specified output directory
model = AutoModelForCausalLM.from_pretrained('results/checkpoint-2805')  # replace 'xxxx' with the appropriate checkpoint
#tokenizer = AutoTokenizer.from_pretrained('results/checkpoint-2805')

# Example math question from your dataset
question = "What is 2 + 8?"  # replace with a real question from the eval_dataset

# Encode the question using the tokenizer
inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Generate the answer using the model
outputs = model.generate(inputs['input_ids'], max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Question: {question}")
print(f"Generated Answer: {answer}")


Question: What is 2 + 8?
Generated Answer: What is 2 + 8? = +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +8 +


In [ ]:
#The End